# MODULES

## Multi-layer perceptrons 

The `vschaos.modules` package provides high-level methods implementing neural networks used in black-box variational inference. Most modules take into arguments two different dictionaries : one regarding the input properties, and one regarding the hidden properties. By example, the :py:class:`MLP` module allows to easily define multi-layered perceptrons (MLP) :


In [ ]:
from vschaos.modules import MLP
import torch

input_params = {'dim':200}
hidden_params = {'nlayers':3, 'dim':[200, 100, 50], 'dropout':0.5, 'batch_norm':'batch'}
module = MLP(input_params, hidden_params)
print('input : ', input_params)
print('hidden : ', hidden_params)
print(module)

x = torch.Tensor(64, 200)
out = module(x)
print(x.shape, '->', out.shape)

In [ ]:
from vschaos.modules import MLP
import torch

# the parameters of each layer can be defined, provided that each arrays' length equals the number of layers
input_params = {'dim':200}
hidden_params = {'nlayers':3, 'dim':[200, 100, 50], 'dropout':[0.0, 0.3, 0.5], 'batch_norm':[None, None, 'batch']}
module = MLP(input_params, hidden_params)
print('input : ', input_params)
print('hidden : ', hidden_params)
print(module)

x = torch.Tensor(64, 200)
out = module(x)
print(x.shape, '->', out.shape)

In [ ]:
import sys; sys.path.append('../..')
from vschaos.modules import MLP
import torch

# The :class:`MLP` also allows multi-input and multi-output
input_params = {'dim':200}
hidden_params = [{'nlayers':3, 'dim':[200, 100, 50], 'dropout':[0.0, 0.3, 0.5], 'batch_norm':[None, None, 'batch']},
                 {'nlayers':2, 'dim':[200, 100], 'dropout':[0.3, 0.5], 'batch_norm':[None, 'batch']}]
module = MLP(input_params, hidden_params, linked=True) # if linked is true, the separate modules concatenate the inputs
print('input : ', input_params)
print('hidden : ', hidden_params)
print(module)

x = torch.Tensor(64, 200)
out = module(x)
print(x[0].shape, '-> ', out[0].shape, ' ; ', x[1].shape, ' -> ', out[1].shape)

In [ ]:
import sys; sys.path.append('../..')
from vschaos.modules import MLP
from vschaos import distributions as dist
import torch

# input conditioning is parametrized in the hidden_params argument, where the label parameters are defined as dicts
input_params = [{'dim':200}, {'dim':100}]
class_params = {'dim':8, 'dist':dist.Categorical}
style_params = {'dim':4, 'dist':dist.Categorical}
hidden_params = {'dim':[200, 30], 'nlayers':2, 'linked':False, 'label_params':{'class':class_params, 'style':style_params}}
module = MLP(input_params, hidden_params)
x = [torch.randn(64, 200), torch.randn(64, 100)]
y = {'class':torch.randint(0, 8, (64,)), 'style':torch.randint(0, 4, (64,))}
print(module)

out = module(x, y=y)
print([x[0].shape, x[1].shape], "->", out.shape)

## Convolutional modules

Besides MLPs, `vschaos` also provides high level methods for convolutional modules, including flattening options to link a latent space to convolutional outputs.



In [ ]:
import torch
from vschaos.modules import MLP
from vschaos.modules import Convolutional, Deconvolutional

# CONVs : convolutional and deconvolutional modules take extended parameters
input_params = {'dim':2048, 'channels':16}
class_params = {'dim':8, 'dist':dist.Categorical}
style_params = {'dim':4, 'dist':dist.Categorical}
hidden_params = {'kernel_size':[16, 8, 4], 'channels':[16, 8, 4], 'stride':[1,2,4], 'dilation':[2,2,4],
                 'batch_norm_conv':['batch', 'batch', 'batch'], 'dropout_conv':[0.2, 0.4, 0.6],
                 'label_params':{'class':class_params, 'style':style_params}, 'conditioning':'concat'}
conv_module = Convolutional(input_params, hidden_params)
deconv_module = Deconvolutional(input_params, hidden_params)
print('conv : ', conv_module)
print('deconv : ', deconv_module)

x = torch.Tensor(64, 16, 1024)
y = {'class':torch.randint(0, 8, (64,)), 'style':torch.randint(0, 4, (64,))}
out = conv_module(x, y=y)
out_deconv = deconv_module(x, y=y)
print('conv', out.shape)
print('deconv', out_deconv.shape)

The `ConvolutionalLatent` class embeds the output convolutional module into a flattened hidden representation of the input with an MLP, allowing to be then projected in a latent space. Inversely, `DeconvolutionalLatent` processes the latent code with an MLP, whose output is resized to be then processed by a `Deconvolution` module. These modules also allow multi-inputs, concatenating hidden representations. 
For `DeconvolutionLatent`, a target size can be given by the pout keyword, allowing to automatically retrieve the output size of the flattening module. **However**, remind that DeconvolutionalLatent does not perform the last convolution from hidden layers to pouts, as this operation is usually performed by a distribution module. See below.

In [ ]:
import torch
from vschaos import distributions as dist
from vschaos.modules import ConvolutionalLatent, DeconvolutionalLatent

input_params = [{'dim':2048, 'channels':16}, {'dim':512, 'channels':16}]
class_params = {'dim':8, 'dist':dist.Categorical}
style_params = {'dim':4, 'dist':dist.Categorical}
hidden_params = {'dim':800, 'nlayers':2, 'kernel_size':[16, 8, 4], 'channels':[16, 8, 4], 'stride':[1,2,4], 'dilation':[2,2,4],
                 'batch_norm_conv':['batch', 'batch', 'batch'], 'dropout_conv':[0.2, 0.4, 0.6],
                 'label_params':{'class':class_params, 'style':style_params}, 'conditioning':'concat'}
conv_module = ConvolutionalLatent(input_params, hidden_params)
deconv_module = DeconvolutionalLatent({'dim':hidden_params['dim'], 'nlayers':1}, hidden_params, pouts=input_params)
print('conv : ', conv_module)
print('deconv : ', deconv_module)

x = torch.Tensor(64, 16, 2048)
x = [torch.Tensor(64, 16, 2048), torch.Tensor(64, 16, 512)]

y = {'class':torch.randint(0, 8, (64,)), 'style':torch.randint(0, 4, (64,))}
out = conv_module(x, y=y)
print('conv', out.shape)

out_deconv = deconv_module(out, y=y)
print('deconv', out_deconv.shape)

## Distribution modules

The `vschaos` library also defines a list of modules that directly encode the parameters of a given distribution, providing directly a `vschaos.Distribution` object. `vschaos.Distribution`, that overrides the native `torch.distribution` library, implements several distribution manipulation functions such as reshaping, squeezing, and defines several other distributions such as random processes.

In [ ]:
import vschaos.distributions as dist 
from vschaos.modules import get_module_from_density

input_params = {'dim': 200}
output_params = {'dim':32, 'dist':dist.Normal} #implemented so far : dist.Bernoulli, dist.Normal, dist.Categorical

# the get_module_from_density returns the correct layer for a given distribution
dist_module = get_module_from_density(output_params['dist'])
module = dist_module(input_params, output_params)
print(module)

x = torch.Tensor(64, 200)
out = module(x)
print(out)

These distribution modules can also be defined using convolution modules, that is automatically done if the input dimensionality is greater than 1 or if the keyword `"conv"` is specified in the output parameters

In [ ]:
import sys; sys.path.append('../..')
import vschaos.distributions as dist 
from vschaos.modules import get_module_from_density

input_params = {'dim': 200, 'channels':[32, 32], 'kernel_size':[5, 3]}
output_params = {'dim': 32, 'dist':dist.Bernoulli, 'conv':True}

# the get_module_from_density returns the correct layer for a given distribution
dist_module = get_module_from_density(output_params['dist'])
module = dist_module(input_params, output_params)
print(module)

x = torch.Tensor(64, 32, 200).normal_()
out = module(x)
print(out)

## Encoders and decoders

In variational auto-encoding modules, variational and generative distributions are defined as distributions whose parameters are defined as functions of respectively the input and the latent vector  $$q(\boldsymbol{z|x}) \sim \mathcal{D}(\boldsymbol{\phi}(\mathbf{x}))$$ 
$$p(\boldsymbol{x|z}) \sim \mathcal{D}(\boldsymbol{\theta}(\mathbf{z}))$$

In the `vschaos` library, these modules are implemented using the `HiddenModule` class, that embeds a bottleneck module (MLP or Convolutional) and a distribution module. They can be additionally conditioned by external class information by specifying the parameters of the corresponding labels.

In [ ]:
import torch
from vschaos import distributions as dist
from vschaos.modules import HiddenModule, ConvolutionalLatent, DeconvolutionalLatent
from vschaos.utils import apply_method

# here we defined two different inputs
input_params = [{'dim':1024, 'channels':2, 'dist':dist.Normal, 'conv':True}, {'dim':2048, 'channels': 2, 'dist':dist.Bernoulli, 'conv':True}]
# the parameters of the conditioning label information
class_params = {'dim':8, 'dist':dist.Categorical}
style_params = {'dim':4, 'dist':dist.Categorical}
# we define the parameters of the bottleneck module
hidden_params = {'dim':800, 'nlayers':2, 'kernel_size':[16, 8, 4], 'channels':[16, 8, 4], 'stride':[1,2,4], 'dilation':[2,2,4], 
                 'batch_norm_conv':['batch', 'batch', 'batch'], 'dropout_conv':[0.2, 0.4, 0.6],
                 'label_params':{'class':class_params, 'style':style_params}, 'conditioning':'concat',
                 'class':ConvolutionalLatent}
# and, finally, the latent parameters (can be split, as we do here in two different latent spaces)
latent_params = [{'dim':8, 'dist':dist.Normal}, {'dim':4, 'dist':dist.Normal}]
encoder = HiddenModule(input_params, phidden=hidden_params, pouts=latent_params)

# forward!
x = [torch.zeros(64, 2, 1024).normal_(), torch.zeros(64, 2, 2048).normal_()]
y = {'class':torch.randint(0, 8, (64,)), 'style':torch.randint(0, 4, (64,))}
out = encoder(x, y=y, return_hidden=True)

print(out.keys())
print('encoder out hidden', out['hidden'][0].shape)
print('encoder out parameters : ', out['out_params'])

# the apply_method allows to apply a method for each element of an array
encoder_out = apply_method(out['out_params'], 'rsample');
print('sampled output : ', encoder_out[0].shape, encoder_out[1].shape)

In [ ]:
# if the module has to mirror an encoder, it can be given to DeconvolutionalLatent
#   to automatically fit the encoder's shape
hidden_params = dict(hidden_params); hidden_params['class'] = DeconvolutionalLatent
decoder = HiddenModule(latent_params, phidden=hidden_params, pouts=input_params, encoder=encoder.hidden_modules)
# print(encoder, decoder)

out = decoder(encoder_out, y=y)
print('decoder out parameters : ', out['out_params'])